### Basic approach - up to 10k elements

In [ ]:
from elasticsearch import Elasticsearch

# Connect to the Elasticsearch cluster
client = Elasticsearch(
    "http://192.168.5.188:9200/", 
    basic_auth=("despot.markovic", "KJmhyg7cPxfVG84UTuE")
)

# Perform a search query on the specified index (insightful-fragments-v6)
response = client.search(
    index="insightful-fragments-v6",
    size=10000,
    query={"match_all": {}}
)

# Access and print the retrieved documents
for doc in response['hits']['hits']:
    print(doc['_source'])

### Scrolling approach - All data pulling

In [ ]:
from elasticsearch import Elasticsearch
import time

# Connect to the Elasticsearch cluster
client = Elasticsearch(
    "http://192.168.5.188:9200/", 
    basic_auth=("despot.markovic", "KJmhyg7cPxfVG84UTuE")
)

# To store the pulled data
all_documents = []

# Start the initial search request with scroll
scroll_size = 10000
scroll_time = '2m'
iteration = 0
total_docs = 0
batch_size = 20

start_time = time.time()

# Initial search query with scroll
response = client.search(
    index="insightful-fragments-v6",
    size=scroll_size,
    scroll=scroll_time,
    query={"match_all": {}}
)

# Extract the first scroll ID and documents
scroll_id = response['_scroll_id']
hits = response['hits']['hits']
total_docs += len(hits)

# Add the first batch of documents to the list
all_documents.extend([doc['_source'] for doc in hits])

# Continue scrolling to pull all data
while len(hits) > 0:
    iteration += 1
    # Fetch the next batch of results using the scroll ID
    response = client.scroll(scroll_id=scroll_id, scroll=scroll_time)
    
    # Update scroll ID and retrieve the next batch of hits
    scroll_id = response['_scroll_id']
    hits = response['hits']['hits']
    total_docs += len(hits)
    
    # Add the new batch of documents to the list
    all_documents.extend([doc['_source'] for doc in hits])
    
    # Print progress every 20 iterations
    if iteration % batch_size == 0:
        time_spent = time.time() - start_time
        print(f"Iteration: {iteration}, Total documents pulled: {total_docs}")
        print(f"Time spent for last 20 iterations: {time_spent:.2f} seconds")
        # Reset the timer after each 20 iterations
        start_time = time.time()

# Once all data is retrieved, clear the scroll context
client.clear_scroll(scroll_id=scroll_id)

print(f"All documents pulled. Total: {total_docs}")
